# A Gentle Introduction to RAG Applications

This notebook creates a simple RAG (Retrieval-Augmented Generation) system to answer questions from a PDF document using an open-source model.

In [1]:
PDF_FILE = "macunaima.pdf"

# We'll be using Llama 3.1 8B for this example.
MODEL = "llama3.1"

## Loading the PDF document

Let's start by loading the PDF document and breaking it down into separate pages.

<img src='images/documents.png' width="1000">

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

pages = pages[:8]

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 8
Length of a page: 1915
Content of a page:  
I  
Macunaíma  
 
No fundo do mato -virgem nasceu Macunaíma, herói de nossa 
gente. Era preto retinto e filho do medo da noite. Houve um momento 
em que o silênci o foi tão grande escutando o murmurejo do 
Uraricoera, que a índia, tapanhumas pariu uma criança feia. Essa 
criança é que chamaram de Macunaíma.  
Já na meninice fez coisas de sarapantar. De primeiro: passou mais 
de seis anos não falando. Sio inci tavam a fal ar exclamava: If — Ai! 
que preguiça!. . . e não dizia mais nada."] Ficava no canto da ma loca, 
trepado no jirau de paxiúba, espiando o trabalho dos outros e 
principalmente os dois manos que tinha, Maanape já velhinho e Jiguê 
na força de homem. O divertimen to dele era decepar cabeça de saúva. 
Vivia deitado mas si punha os olhos em dinheiro, Macunaí ma dandava 
pra ganhar vintém. E também espertava quando a família ia tomar 
banho no rio, todos juntos e nus. Passava o tempo do banho dando 
mergulho, e a

## Splitting the pages in chunks

Pages are too long, so let's split pages into different chunks.

<img src='images/splitter.png' width="1000">


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=300)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 8
Length of a chunk: 1911
Content of a chunk: I  
Macunaíma  
 
No fundo do mato -virgem nasceu Macunaíma, herói de nossa 
gente. Era preto retinto e filho do medo da noite. Houve um momento 
em que o silênci o foi tão grande escutando o murmurejo do 
Uraricoera, que a índia, tapanhumas pariu uma criança feia. Essa 
criança é que chamaram de Macunaíma.  
Já na meninice fez coisas de sarapantar. De primeiro: passou mais 
de seis anos não falando. Sio inci tavam a fal ar exclamava: If — Ai! 
que preguiça!. . . e não dizia mais nada."] Ficava no canto da ma loca, 
trepado no jirau de paxiúba, espiando o trabalho dos outros e 
principalmente os dois manos que tinha, Maanape já velhinho e Jiguê 
na força de homem. O divertimen to dele era decepar cabeça de saúva. 
Vivia deitado mas si punha os olhos em dinheiro, Macunaí ma dandava 
pra ganhar vintém. E também espertava quando a família ia tomar 
banho no rio, todos juntos e nus. Passava o tempo do banho dando 
mergulho, e 

## Storing the chunks in a vector store

We can now generate embeddings for every chunk and store them in a vector store.

<img src='images/vectorstore.png' width="1000">


In [5]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = FAISS.from_documents(chunks, embeddings)

## Setting up a retriever

We can use a retriever to find chunks in the vector store that are similar to a supplied question.

<img src='images/retriever.png' width="1000">



In [6]:
retriever = vectorstore.as_retriever()
retriever.invoke("Quem é Macunaíma?")

[Document(metadata={'source': 'macunaima.pdf', 'page': 5}, page_content='Quando Jiguê não pôde mais surrar, Macunaíma correu até a \ncapoeira, mastigou  raiz de cardeiro e vol tou são.  Jiguê levou Sofará \npro pai dela e dormiu fol gado na  rede.'),
 Document(metadata={'source': 'macunaima.pdf', 'page': 1}, page_content='I  \nMacunaíma  \n \nNo fundo do mato -virgem nasceu Macunaíma, herói de nossa \ngente. Era preto retinto e filho do medo da noite. Houve um momento \nem que o silênci o foi tão grande escutando o murmurejo do \nUraricoera, que a índia, tapanhumas pariu uma criança feia. Essa \ncriança é que chamaram de Macunaíma.  \nJá na meninice fez coisas de sarapantar. De primeiro: passou mais \nde seis anos não falando. Sio inci tavam a fal ar exclamava: If — Ai! \nque preguiça!. . . e não dizia mais nada."] Ficava no canto da ma loca, \ntrepado no jirau de paxiúba, espiando o trabalho dos outros e \nprincipalmente os dois manos que tinha, Maanape já velhinho e Jiguê \nna força 

## Configuring the model

We'll be using Ollama to load the local model in memory. After creating the model, we can invoke it with a question to get the response back.

<img src='images/model.png' width="1000">

In [7]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Quem foi Pedro Alvares Cabral?")

AIMessage(content='Pedro Álvares Cabral foi um navegador português que liderou uma expedição que descobriu o Brasil em 1500.', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-06T14:00:01.8449084Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 9897712900, 'load_duration': 45848100, 'prompt_eval_count': 20, 'prompt_eval_duration': 1324277000, 'eval_count': 33, 'eval_duration': 8523105000}, id='run-2e1f162c-7aed-424b-88d3-b93b03c8ba0f-0', usage_metadata={'input_tokens': 20, 'output_tokens': 33, 'total_tokens': 53})

## Parsing the model's response

The response from the model is an `AIMessage` instance containing the answer. We can extract the text answer by using the appropriate output parser. We can connect the model and the parser using a chain.

<img src='images/parser.png' width="1000">


In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Quem foi o presidente do Brasil em 1963"))

Em 1963, o presidente do Brasil era João Goulart.


## Setting up a prompt

In addition to the question we want to ask, we also want to provide the model with the context from the PDF file. We can use a prompt template to define and reuse the prompt we'll use with the model.


<img src='images/prompt.png' width="1000">

In [10]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "Desculpe. Eu não sei!".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="O contexto será interpolado aqui", question="Aqui será interpolada a questão"))


You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: O contexto será interpolado aqui

Question: Aqui será interpolada a questão



## Adding the prompt to the chain

We can now chain the prompt with the model and the parser.

<img src='images/chain1.png' width="1000">

In [11]:
chain = prompt | model | parser

chain.invoke({
    "context": "João é filho de Paulo, Paulo é irmão de Fernando, Fernando possui dois filhos: Dudu e Carina. ", 
    "question": "O que João é de Carina?"
})


'Tio.'

## Adding the retriever to the chain

Finally, we can connect the retriever to the chain to get the context from the vector store.

<img src='images/chain2.png' width="1000">

In [12]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

## Using the chain to answer questions

Finally, we can use the chain to ask questions that will be answered using the PDF document.

In [16]:
questions = [    
    "Em algum momento macunaíma chorou? Se sim pode me citar estes momentos.",    
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: Em algum momento macunaíma chorou? Se sim pode me citar estes momentos.
Answer: Sim, Macunaíma chorou em alguns momentos. Aqui estão os momentos em que ele chorou:

1.  Quando a mãe dele não quis largar da mandioca para levá-lo passear no mato. Ele chorou dia inteiro e também à noite.
2.  Quando pediu pra nora, companheira de Jiguê, que levasse o menino, mas ela foi até o pé de aninga na beira do rio e ele ficou triste porque tinha muita formiga!
3.  Quando a moça botou Macunaíma na praia, ele principiou choramingando.
4.  Quando Jiguê não desconfiou de nada e começou trançando corda com fibra de curauá, Macunaíma pediu um pedaço de curauá pro ma no porém Jiguê falou que aquilo não era brinquedo de criança.
*************************

